<a href="https://colab.research.google.com/github/yeop6134/Ai_humanities/blob/main/BERT%EA%B8%B0%EB%B0%98TopicModeling/Korean_CTM_%EB%AA%85%EC%82%AC%EC%82%AC%EC%9A%A9_%EB%B6%88%EC%9A%A9%EC%96%B4%EC%A0%9C%EA%B1%B0_newDiscrimination_Okt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install contextualized-topic-models==2.2.0
!pip install pyldavis

# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/인공지능인문학"

/content/drive/MyDrive/Colab Notebooks/인공지능인문학


In [ ]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab, Okt
from tqdm import tqdm

In [ ]:
documents = [line.strip() for line in open("newDiscrimination_text.txt", encoding="utf-8").readlines()]

In [ ]:
documents[:5]

['못사는 놈들은 정말 불쌍해.',
 '쟤처럼 느글거리는 인상이 꼭 사고치더라',
 '어린 애들 따 먹어봤다는 거잖아 좀 부럽네',
 '너 계속 이러면 나중에 청소차 몰고 다닌다고 엄마가 말했어 안했어',
 '와 미국인 정서 걍 이해불가던데 어떻게 사귀는거지?']

In [ ]:
korean_stopwords_path = "./korean_stopwords.txt"

# 텍스트 파일을 오픈합니다.
with open(korean_stopwords_path, encoding='utf-8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

In [ ]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.nouns(sent)  # 명사만 토큰화
        words = [word for word in word_tokens if len(word) > 1]

        result = []
        for w in words: 
         if w not in stopwords: 
            result.append(w)

        return result

In [ ]:
custom_tokenizer = CustomTokenizer(Okt())

In [ ]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [ ]:
train_bow_embeddings = vectorizer.fit_transform(documents)

In [ ]:
print(train_bow_embeddings.shape)

(4334, 3000)


In [ ]:
vocab = vectorizer.get_feature_names()
id2token = {k: v for k, v in zip(range(0, len(vocab)), vocab)}

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
len(vocab)

3000

In [ ]:
train_contextualized_embeddings = bert_embeddings_from_list(documents, \
                                                            "sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

In [ ]:
qt = TopicModelDataPreparation()

training_dataset = qt.load(train_contextualized_embeddings, train_bow_embeddings, id2token)

# Combined TM 학습하기

In [ ]:
ctm = CombinedTM(bow_size=len(vocab), contextual_size=768, n_components=8, num_epochs=20)
ctm.fit(training_dataset)

Epoch: [20/20]	 Seen Samples: [86680/86680]	Train Loss: 22.220477968083035	Time: 0:00:01.734698: : 20it [00:29,  1.48s/it]


# 토픽들
학습 후에는 토픽 모델이 선정한 토픽들을 보려면 아래의 메소드를 사용합니다.  


> get_topic_lists


해당 메소드에는 각 토픽마다 몇 개의 단어를 보고 싶은지에 해당하는 파라미터를 넣어즐 수 있습니다.

In [ ]:
ctm.get_topics(10)

defaultdict(list,
            {0: ['중국',
              '조선족',
              '흑인',
              '중국인',
              '동남아',
              '북한',
              '한국',
              '냄새',
              '일본인',
              '한국어'],
             1: ['장애인', '나라', '사람', '장애', '이슬람', '부모', '학교', '공부', '사회', '친구'],
             2: ['회사', '인정', '얼굴', '아파트', '운전', '주제', '김여사', '집안', '알바', '거임'],
             3: ['페미', '얼굴', '커피', '한남', '화장', '아이돌', '시집', '소방관', '나이', '스시'],
             4: ['여자', '결혼', '여성', '여자애', '요즘', '남자', '페미', '무조건', '나이', '쯧쯧'],
             5: ['짱깨',
              '건가',
              '다이어리',
              '수저',
              '홍어',
              '후진국',
              '서민',
              '지금',
              '공기업',
              '노예'],
             6: ['명함', '요트', '엄마', '노릇', '이집', '다리', '작용', '정신병원', '지면', '적용'],
             7: ['남자',
              '원래',
              '애가',
              '게이',
              '한남',
              '아들',
              '결혼',
              '남성',
  

# 시각화

In [ ]:
%pip install pyLDAvis

In [ ]:
import pyLDAvis as vis

lda_vis_data = ctm.get_ldavis_data_format(vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)
     

/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping
Sampling: [10/10]: : 10it [00:16,  1.61s/it]
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


## 에포크 높여서(50) 돌려보기

In [ ]:
ctm = CombinedTM(bow_size=len(vocab), contextual_size=768, n_components=8, num_epochs=50)
ctm.fit(training_dataset)

Epoch: [50/50]	 Seen Samples: [216700/216700]	Train Loss: 21.811594753150956	Time: 0:00:01.345833: : 50it [01:23,  1.67s/it]


In [ ]:
ctm.get_topics(10)

defaultdict(list,
            {0: ['남자', '애가', '얼굴', '화장', '여자', '결혼', '정도', '나이', '한남', '게이'],
             1: ['여자', '결혼', '여성', '머리', '남자', '무조건', '여자애', '군대', '운전', '다운'],
             2: ['짱깨', '아가씨', '감탄', '쪽바리', '짐꾼', '카레', '수저', '돼지', '진흙', '여경'],
             3: ['장애인',
              '흑인',
              '장애',
              '정신병',
              '차별',
              '게이',
              '환자',
              '문제',
              '냄새',
              '정상인'],
             4: ['공부', '회사', '부모', '교육', '학교', '주제', '집안', '가정', '아파트', '임대'],
             5: ['나라',
              '조선족',
              '서울',
              '한국',
              '동네',
              '우리나라',
              '난민',
              '한국인',
              '시골',
              '한국어'],
             6: ['면서', '백인', '대로', '모함', '무기', '실수', '감탄', '잔소리', '비만', '듣기'],
             7: ['중국',
              '중국인',
              '사람',
              '동양인',
              '동남아',
              '아시아',
              '서양인',
              '일본',
  